In [0]:
processor.add_placeholder(
    "age",
    f'DATEDIFF(DATE("{base_date}"), person_birth_date) / 365.25 AS person_age',
    None,
    None,
)

processor.add_placeholder(
    "forward_65",
    "f65cte.forward_65_flag",
    cte_sql=f"""
        forward_65_cte AS (
            SELECT
                person_id,
                CASE WHEN coalesce(pcp_name, '') IN (
                    SELECT pcp_name
                    FROM geisinger.reference_data.physicians_65_forward
                    WHERE active65F = 1
                ) THEN 'Y'
                ELSE 'N' END AS forward_65_flag
            FROM {gbl_target}patients
        )
    """,
    join_sql="LEFT JOIN forward_65_cte f65cte ON patients.person_id = f65cte.person_id",
)

processor.add_placeholder(
    "ghp_gold",
    f"""
        CASE WHEN insurance_category.plan_category = 'GHP Gold' THEN 'Y'
        ELSE 'N' END AS ghp_gold
    """,
    None,
    None,
)

processor.add_placeholder(
    "ghp_gold_staying",
    'coalesce(ghpgt.ghp_gold_staying, "Y") as ghp_gold_staying',
    cte_sql=f"""
        address_id_added AS (
            SELECT
                person_id, 
                sha1(concat_ws('|',
                    coalesce(lower(person_mailing_add_line_1), ''), 
                    coalesce(lower(person_mailing_add_line_2), ''),
                    coalesce(lower(person_mailing_city), ''), 
                    coalesce(lower(person_mailing_state), ''),
                    coalesce(person_mailing_zip, '')
                    )) AS address_id
            FROM {gbl_target}patients
        ),
        ghp_gold_transition_cte AS (
            SELECT a.person_id, ins.plan_name, add.address_county, 'N' as ghp_gold_staying
            FROM address_id_added a 
            JOIN {gbl_target}insurance ins ON ins.person_id = a.person_id 
            JOIN geisinger.reference_data.geisinger_address add ON a.address_id = add.address_id 
            WHERE ins.plan_name = 'GHP GOLD PREFERRED ENHANCED MP-DD' or
            (ins.plan_name = 'GHP GOLD CLASSIC 360 RX MH-1D' and add.address_county in 
                ('Schuylkill', 'York', 'Carbon', 'Wayne',  'Bradford', 'Adams', 'Lebanon', 'Franklin', 'Cumberland', 'Dauphin', 'Perry'))
        )
    """,
    join_sql="LEFT JOIN ghp_gold_transition_cte ghpgt ON patients.person_id = ghpgt.person_id",
)

processor.add_placeholder(
    "turning_65",
    f"""
        CASE WHEN DATEDIFF(DATE("{base_date}"), person_birth_date) / 365.25 < 65.0 THEN 'Y' 
        ELSE 'N' END AS turning_65
    """,
    None,
    None,
)

processor.add_placeholder(
    "65_plus",
    f"""
        CASE WHEN DATEDIFF(DATE("{base_date}"), person_birth_date) / 365.25 >= 65.0 THEN 'Y' 
        ELSE 'N' END AS 65_plus
    """,
    None,
    None,
)

processor.add_placeholder(
    "medications_last_year",
    'coalesce(g1cte.medications_last_year, "0 - Unknown") as medications_last_year',
    cte_sql=f"""
        medications_last_year_cte AS (
            SELECT
            CASE 
                WHEN coalesce(count(distinct(medication_id)), 0) <= 2 THEN '1 - Low'
                WHEN coalesce(count(distinct(medication_id)), 0) between 2 and 12 THEN '2 - Moderate'
                WHEN coalesce(count(distinct(medication_id)), 0) >=13 THEN '3 - High' 
            END as medications_last_year, person_id
            FROM {gbl_target}medications 
            WHERE rx_date_ordered >= add_months(DATE("{max_encounter_date}"), -12)
            GROUP BY person_id
        )
    """,
    join_sql="LEFT JOIN medications_last_year_cte g1cte ON patients.person_id = g1cte.person_id",
)

processor.add_placeholder(
    "hospital_encounters",
    'coalesce(g2cte.hospital_encounters, "0 - Unknown") as hospital_encounters',
    cte_sql=f"""
        hospital_encounters_cte AS (
            SELECT 
            CASE 
                WHEN count(distinct(encounter_id)) <= 1 THEN '1 - Low'
                WHEN count(distinct(encounter_id)) between 2 and 4 THEN '2 - Moderate'
                WHEN count(distinct(encounter_id)) >4 THEN '3 - High'
            END AS hospital_encounters, person_id
            FROM {gbl_target}encounters
            WHERE encounter_date >= add_months(DATE("{max_encounter_date}"), -24)
            AND encounter_type = "Hospital Encounter"
            GROUP BY person_id
        )
    """,
    join_sql="LEFT JOIN hospital_encounters_cte g2cte ON patients.person_id = g2cte.person_id",
)

processor.add_placeholder(
    "all_encounters",
    'coalesce(g3cte.all_encounters, "0 - Unknown") as all_encounters',
    cte_sql=f"""
        all_encounters_cte AS (
            SELECT 
            CASE 
                WHEN count(distinct(encounter_id)) <= 3 THEN '1 - Low'
                WHEN count(distinct(encounter_id)) between 4 and 10 THEN '2 - Moderate'
                WHEN count(distinct(encounter_id)) >10 THEN '3 - High'
            END AS all_encounters, person_id
            FROM {gbl_target}encounters
            WHERE encounter_date >= add_months(DATE("{max_encounter_date}"), -24)
            GROUP BY person_id
        )
    """,
    join_sql="LEFT JOIN all_encounters_cte g3cte ON patients.person_id = g3cte.person_id",
)

processor.add_placeholder(
    "facility_address_id_65f",
    "g4cte.facility_address_id_65f",
    cte_sql=f"""
    patients_address_id_65f AS (
        SELECT person_id, 
            sha1(concat_ws('|',
                coalesce(lower(person_mailing_add_line_1), ''), 
                coalesce(lower(person_mailing_add_line_2), ''),
                coalesce(lower(person_mailing_city), ''), 
                coalesce(lower(person_mailing_state), ''),
                coalesce(person_mailing_zip, '')
                )) AS address_id
        FROM {gbl_target}patients
    ),
    facilities_with_distance_65f AS (
        SELECT 
            p.person_id, 
            f.address_id AS facility_address_id,
            2 * 3958.8 * ASIN(
                SQRT(
                    POWER(SIN(RADIANS(f.address_latitude - a.address_latitude) / 2), 2) +
                    COS(RADIANS(a.address_latitude)) * COS(RADIANS(f.address_latitude)) *
                    POWER(SIN(RADIANS(f.address_longitude - a.address_longitude) / 2), 2)
                )
            ) AS distance
        FROM patients_address_id_65f p
        LEFT JOIN geisinger.reference_data.geisinger_address a
            ON p.address_id = a.address_id
        JOIN geisinger.reference_data.geisinger_facilities f 
            WHERE f.65_forward_facility = 'Y'
            AND f.address_closed = 'N'
            AND f.address_id != '65 Forward Health Center Milton'
    ),
    ranked_facilities_65f AS (
        SELECT 
            person_id, facility_address_id, distance,
            ROW_NUMBER() OVER (PARTITION BY person_id ORDER BY distance ASC) AS rank
        FROM facilities_with_distance_65f
    ),
    closest_facility_65f_cte AS (
        SELECT 
            person_id,
            MAX(CASE WHEN rank = 1 THEN facility_address_id END) AS facility_address_id_65f,
            MAX(CASE WHEN rank = 1 THEN coalesce(distance, 0) END) AS facility_distance_65f
        FROM ranked_facilities_65f
        WHERE rank <= 3
        GROUP BY person_id
    )
    """,
    join_sql="LEFT JOIN closest_facility_65f_cte g4cte ON patients.person_id = g4cte.person_id",
)

processor.add_placeholder(
    "facility_distance_65f", "g4cte.facility_distance_65f", None, None
)

processor.add_placeholder(
    "facility_address_id",
    "g5cte.facility_address_id",
    cte_sql=f"""
        patients_address_id AS (
        SELECT person_id, 
            sha1(concat_ws('|',
                coalesce(lower(person_mailing_add_line_1), ''), 
                coalesce(lower(person_mailing_add_line_2), ''),
                coalesce(lower(person_mailing_city), ''), 
                coalesce(lower(person_mailing_state), ''),
                coalesce(person_mailing_zip, '')
                )) AS address_id
        FROM {gbl_target}patients
    ), 
    patients_with_h3 AS (
        SELECT a.*, b.address_h3_point
        FROM patients_address_id a
        JOIN geisinger.reference_data.geisinger_address b 
            ON a.address_id = b.address_id
    ),
    facilities_with_distance AS (
        SELECT 
            p.person_id, 
            f.address_id AS facility_address_id,
            f.address_h3_point,
            h3_try_distance(p.address_h3_point, f.address_h3_point) AS distance
        FROM patients_with_h3 p
        JOIN geisinger.reference_data.geisinger_facilities f 
            ON p.address_h3_point IS NOT NULL 
            AND f.address_h3_point IS NOT NULL
            AND f.address_closed = 'N'
            AND f.address_type in ('Cancer center',
                'Emergency department',
                'Hospital',
                'Primary care clinic',
                'Senior care',
                'Specialty care clinic',
                'Urgent care clinic'
            ) 
    ),
    ranked_facilities AS (
        SELECT 
            person_id, facility_address_id, address_h3_point, distance,
            ROW_NUMBER() OVER (PARTITION BY person_id ORDER BY distance ASC) AS rank
        FROM facilities_with_distance
    ),
    closest_facility_cte AS (
        SELECT 
            person_id,
            MAX(CASE WHEN rank = 1 THEN facility_address_id END) AS facility_address_id,
            MAX(CASE WHEN rank = 1 THEN coalesce(distance, 0) END) AS facility_distance
        FROM ranked_facilities
        WHERE rank <= 3
        GROUP BY person_id
    )
    """,
    join_sql="LEFT JOIN closest_facility_cte g5cte ON patients.person_id = g5cte.person_id",
)

processor.add_placeholder("facility_distance", "g5cte.facility_distance", None, None)

processor.add_placeholder(
    "single_person_household",
    f"""
    CASE WHEN acxiom_features.IBE7629_01 IS NULL THEN 'N'
        WHEN acxiom_features.IBE7629_01 = '1' THEN 'Y'
        ELSE 'N' END AS single_person_household
    """,
    None,
    None,
)

processor.add_placeholder(
    "interested_in_health_fitness",
    f"""
    CASE WHEN acxiom_features.TP000653 <= 69 THEN 1 ELSE 0 END AS iihf1,
    CASE WHEN acxiom_features.AP006251_01 >= 36 THEN 1 ELSE 0 END AS iihf2,
    CASE WHEN acxiom_features.TP001697 <= 48 THEN 1 ELSE 0 END AS iihf3,
    CASE WHEN acxiom_features.IBE7288 = 1 THEN 1 ELSE 0 END AS iihf4,
    CASE WHEN iihf1 + iihf2 + iihf3 + iihf4 > 2 THEN 'Y' ELSE 'N' END AS interested_in_health_fitness
    """,
    None,
    None,
)

processor.add_placeholder(
    "chronic_condition",
    "coalesce(g6cte.chronic_condition, 'N') AS chronic_condition",
    cte_sql=f"""
        chronic_condition_cte AS (
            SELECT case when count(*) > 0 then 'Y' else 'N' end as chronic_condition, person_id
            FROM {gbl_target}diagnoses 
            JOIN data_science.reference_data.ccir_v_2024_1 on UPPER(REGEXP_REPLACE(diagnoses.diagnosis_code, '\\\\.', '')) = ccir_v_2024_1.icd10_cm_code
            WHERE ccir_v_2024_1.chronic_indicator = 1
            GROUP BY diagnoses.person_id
        )
    """,
    join_sql="LEFT JOIN chronic_condition_cte g6cte ON patients.person_id = g6cte.person_id",
)

processor.add_placeholder(
    "preventative_care",
    "coalesce(g7cte.preventative_screening, 'N') AS preventative_care",
    cte_sql=f"""
        procedure_counts AS (
            SELECT 
                procedures.person_id, 
                YEAR(procedures.contact_date) AS procedure_year,
                COUNT(*) AS procedure_count
            FROM {gbl_target}procedures
            JOIN geisinger.features.preventative_care_codes 
                ON procedures.procedure_code = preventative_care_codes.code
            WHERE YEAR(procedures.contact_date) >= YEAR(CURRENT_DATE) - 3  -- Filter for the last 3 years
            GROUP BY procedures.person_id, YEAR(procedures.contact_date)
        ),
        person_year_check AS (
            SELECT
                person_id,
                procedure_year,
                CASE WHEN procedure_count > 0 THEN 1 ELSE 0 END AS has_procedure
            FROM procedure_counts
        ),
        person_screening_summary AS (
            SELECT
                person_id,
                SUM(has_procedure) AS years_with_procedures
            FROM person_year_check
            GROUP BY person_id
        ),
        preventative_care_cte AS (
            SELECT
                person_id,
                CASE 
                    WHEN years_with_procedures >= 2 THEN 'Y'
                    ELSE 'N'
                END AS preventative_screening
            FROM person_screening_summary
        )
    """,
    join_sql="LEFT JOIN preventative_care_cte g7cte ON patients.person_id = g7cte.person_id",
)

processor.add_placeholder(
    "enrolled_in_MA",
    f"""
        CASE WHEN insurance_category.plan_category in ('GHP Gold', 'GHP DSNP', 'Competing MA') THEN 'Y'
        ELSE 'N' END AS enrolled_in_MA
    """,
    None,
    None,
)

processor.add_placeholder(
    "total_chronic_conditions",
    "coalesce(g8cte.total_chronic_conditions, '0 Conditions') AS total_chronic_conditions",
    cte_sql=f"""
        chronic_condition_count_cte AS (
            SELECT case when count(distinct diagnoses.diagnosis_code) in (1) then '1 Conditions' 
                   when count(distinct diagnoses.diagnosis_code) >= 2 then '2+ Conditions' 
                   else 'No Conditions' end as total_chronic_conditions, person_id
            FROM {gbl_target}diagnoses 
            JOIN data_science.reference_data.ccir_v_2024_1 on UPPER(REGEXP_REPLACE(diagnoses.diagnosis_code, '\\\\.', '')) = ccir_v_2024_1.icd10_cm_code
            WHERE ccir_v_2024_1.chronic_indicator = 1
            GROUP BY diagnoses.person_id
        )
    """,
    join_sql="LEFT JOIN chronic_condition_count_cte g8cte ON patients.person_id = g8cte.person_id",
)

processor.add_placeholder(
    "total_preventative_procedures",
    "coalesce(g9cte.total_preventative_procedures, '0 Procedures') AS total_preventative_procedures",
    cte_sql=f"""
        preventative_procedure_count_cte AS (
            SELECT case when count(procedures.procedure_code) in (1,2) then '1-2 Procedures' 
                   when count(procedures.procedure_code) >= 3 then '3+ Procedures' 
                   else 'No Procedures' end as total_preventative_procedures, person_id
            FROM {gbl_target}procedures 
            JOIN geisinger.features.preventative_care_codes on procedures.procedure_code = preventative_care_codes.code
            GROUP BY procedures.person_id
        )
    """,
    join_sql="LEFT JOIN preventative_procedure_count_cte g9cte ON patients.person_id = g9cte.person_id",
)

processor.add_placeholder(
    "nadac_burden",
    'coalesce(g10cte.nadac_burden, "0 - Unknown") AS nadac_burden',
    cte_sql=f"""
        MaxAsOfDate_nadac AS (
            SELECT 
                ndc, 
                ndc_description, 
                nadac_per_unit, 
                effective_date, 
                pricing_unit, 
                pharmacy_type_indicator, 
                otc, 
                explanation_code, 
                rate_setting_class, 
                generic_equivalent_nadac_per_unit, 
                generic_equivalent_effective_date, 
                as_of_date,
                ROW_NUMBER() OVER (PARTITION BY ndc ORDER BY as_of_date DESC) AS row_num
            FROM 
                data_science.reference_data.nadac_v09_25_2024
        ),
        MedicationData_nadac AS (
            SELECT 
                a.person_id,
                COUNT(DISTINCT a.medication_id) / NULLIF(COUNT(DISTINCT DATE_TRUNC('month', a.rx_date_ordered)), 0) AS average_monthly_medication_load,
                COALESCE(SUM(COALESCE(CAST(REGEXP_EXTRACT(a.quantity, '([0-9]+)') AS DOUBLE), 1) * b.nadac_per_unit), 0) AS total_NADAC,
                COUNT(DISTINCT DATE_TRUNC('month', a.rx_date_ordered)) AS month_count
            FROM 
                {gbl_target}medications a
            LEFT JOIN 
                MaxAsOfDate_nadac b 
            ON 
                NULLIF(TRY_CAST(REPLACE(a.drug_NDC, '-', '') AS BIGINT), 0) = b.ndc
            WHERE 
                b.row_num = 1
            GROUP BY 
                a.person_id
        ),
        NADACDistribution AS (
            SELECT 
                person_id, 
                total_NADAC / NULLIF(month_count, 0) AS average_monthly_nadac,
                NTILE(3) OVER (ORDER BY total_NADAC / NULLIF(month_count, 0)) AS category
            FROM 
                MedicationData_nadac
        ),
        nadac_burden_cte as (
            SELECT 
                person_id, 
                average_monthly_nadac,
                CASE 
                    WHEN category = 1 THEN '1 - Low'
                    WHEN category = 2 THEN '2 - Medium'
                    WHEN category = 3 THEN '3 -High'
                END AS nadac_burden
            FROM 
                NADACDistribution
        )
    """,
    join_sql="LEFT JOIN nadac_burden_cte g10cte ON patients.person_id = g10cte.person_id",
)

processor.add_placeholder(
    "time_weighted_medication_load",
    'coalesce(g11cte.time_weighted_medication_load, "0 - Unknown") AS time_weighted_medication_load',
    cte_sql=f"""
        time_weighted_medication_load_cte AS (
        SELECT 
            person_id,
            CASE 
                WHEN COUNT(DISTINCT medication_id) / COUNT(DISTINCT DATE_TRUNC('month', rx_date_ordered)) <= 2.3 THEN '1 - Low'
                WHEN COUNT(DISTINCT medication_id) / COUNT(DISTINCT DATE_TRUNC('month', rx_date_ordered)) BETWEEN 2.3 AND 5.9 THEN '2 - Moderate'
                WHEN COUNT(DISTINCT medication_id) / COUNT(DISTINCT DATE_TRUNC('month', rx_date_ordered)) > 5.9 THEN '3 - High'
            END AS time_weighted_medication_load 
            FROM {gbl_target}medications
        GROUP BY person_id
        )   
    """,
    join_sql="LEFT JOIN time_weighted_medication_load_cte g11cte ON patients.person_id = g11cte.person_id",
)

processor.add_placeholder(
    "on_maintenance_medication",
    'coalesce(g12cte.on_maintenance_medication, "N") AS on_maintenance_medication',
    cte_sql=f"""
        filtered_maintenance_medication AS (
            SELECT 
                person_id,
                SUBSTRING(drug_NDC, 1, LENGTH(drug_NDC) - 3) AS truncated_NDC,  
                COUNT(order_id) AS order_count
            FROM {gbl_target}medications
            GROUP BY person_id, truncated_NDC
        ),
        maintenance_medication_cte AS (
            SELECT 
                person_id,
                CASE WHEN COUNT(truncated_NDC) > 0 THEN "Y" ELSE "N" END AS on_maintenance_medication
            FROM filtered_maintenance_medication
            WHERE order_count >= 4  
            GROUP BY person_id
        )
    """,
    join_sql="LEFT JOIN maintenance_medication_cte g12cte ON patients.person_id = g12cte.person_id",
)

processor.add_placeholder(
    "maintenance_medication_count",
    "coalesce(g13cte.maintenance_medication_count, 0) AS maintenance_medication_count",
    cte_sql=f"""
        filtered_maintenance_medication_count AS (
            SELECT 
                person_id,
                SUBSTRING(drug_NDC, 1, LENGTH(drug_NDC) - 3) AS truncated_NDC,  
                COUNT(order_id) AS order_count
            FROM {gbl_target}medications
            GROUP BY person_id, truncated_NDC
        ),
        maintenance_medication_count_cte AS (
            SELECT 
                person_id,
                COUNT(truncated_NDC) AS maintenance_medication_count
            FROM filtered_maintenance_medication_count
            WHERE order_count >= 4  
            GROUP BY person_id
        )
    """,
    join_sql="LEFT JOIN maintenance_medication_count_cte g13cte ON patients.person_id = g13cte.person_id",
)

processor.add_placeholder(
    "high_cost_medication",
    'coalesce(g14cte.high_cost_medication, "N") AS high_cost_medication',
    cte_sql=f"""
        MaxAsOfDate_nadac_hc AS (
            SELECT 
                ndc, 
                nadac_per_unit, 
                ROW_NUMBER() OVER (PARTITION BY ndc ORDER BY as_of_date DESC) AS row_num
            FROM data_science.reference_data.nadac_v09_25_2024
        ),
        medications_nadac_hc AS (
            SELECT 
                a.person_id,
                a.medication_id,
                a.drug_NDC,
                b.nadac_per_unit
            FROM {gbl_target}medications a
            LEFT JOIN MaxAsOfDate_nadac_hc b 
            ON NULLIF(TRY_CAST(REPLACE(a.drug_NDC, '-', '') AS BIGINT), 0) = b.ndc
            WHERE b.row_num = 1  
        ),
        high_cost_medication_cte AS (
        SELECT 
            person_id,
            CASE 
                WHEN MAX(nadac_per_unit) >= 10 THEN 'Y'
                ELSE 'N'      
            END AS high_cost_medication
        FROM medications_nadac_hc
        GROUP BY person_id
        )
    """,
    join_sql="LEFT JOIN high_cost_medication_cte g14cte ON patients.person_id = g14cte.person_id",
)

processor.add_placeholder(
    "geisinger_mail_order_user",
    'coalesce(g15cte.geisinger_mail_order_user, "N") AS geisinger_mail_order_user',
    cte_sql=f"""
        geisinger_mail_order_user_cte AS (
            SELECT 
            person_id,
            CASE 
                WHEN COUNT(CASE WHEN fill_pharmacy = 'GEISINGER MAIL ORDER PHARMACY' THEN 1 END) > 0 THEN 'Y' ELSE 'N' 
            END AS geisinger_mail_order_user
        FROM {gbl_target}medications
        GROUP BY person_id
        )
    """,
    join_sql="LEFT JOIN geisinger_mail_order_user_cte g15cte ON patients.person_id = g15cte.person_id",
)

processor.add_placeholder(
    "lapsed_mail_order_user",
    'coalesce(g16cte.lapsed_mail_order_user, "Y") AS lapsed_mail_order_user',
    cte_sql=f"""
        recent_medications_lapsed AS (
            SELECT 
                m.person_id,
                COUNT(1) AS recent_fill_count
            FROM {gbl_target}medications m
            WHERE m.fill_pharmacy = 'GEISINGER MAIL ORDER PHARMACY'
                AND m.rx_date_filled BETWEEN DATE_SUB('{max_encounter_date}', 180) AND '{max_encounter_date}'
            GROUP BY m.person_id
        ),
        prior_medications_lapsed AS (
            SELECT 
                m.person_id,
                COUNT(1) AS prior_fill_count
            FROM {gbl_target}medications m
            WHERE 
                m.fill_pharmacy = 'GEISINGER MAIL ORDER PHARMACY'
                AND m.rx_date_filled < DATE_SUB('{max_encounter_date}', 180)
            GROUP BY m.person_id
        ),
        lapsed_mail_order_user_cte AS (
            SELECT 
                r.person_id,
                CASE WHEN COALESCE(r.recent_fill_count, 0) = 0 AND COALESCE(p.prior_fill_count, 0) > 0 THEN 'Y' ELSE 'N' 
                END AS lapsed_mail_order_user
            FROM recent_medications_lapsed r 
            LEFT JOIN prior_medications_lapsed p 
            ON r.person_id = p.person_id
        )
    """,
    join_sql="LEFT JOIN lapsed_mail_order_user_cte g16cte ON patients.person_id = g16cte.person_id",
)

processor.add_placeholder(
    "within_25m_of_geisinger_pharmacy",
    'coalesce(g17cte.within_25m_of_pharmacy, "N") AS within_25m_of_geisinger_pharmacy',
    cte_sql=f"""
    rx_patients_address_id AS (
        SELECT person_id, 
            sha1(concat_ws('|',
                coalesce(lower(person_mailing_add_line_1), ''), 
                coalesce(lower(person_mailing_add_line_2), ''),
                coalesce(lower(person_mailing_city), ''), 
                coalesce(lower(person_mailing_state), ''),
                coalesce(person_mailing_zip, '')
                )) AS address_id
        FROM {gbl_target}patients
    ), 
    rx_patients_with_h3 AS (
        SELECT a.*, b.address_h3_point
        FROM rx_patients_address_id a
        JOIN geisinger.reference_data.geisinger_address b 
            ON a.address_id = b.address_id
    ),
    rx_facilities_with_distance AS (
        SELECT 
            p.person_id, 
            f.address_id AS facility_address_id,
            f.address_h3_point,
            h3_try_distance(p.address_h3_point, f.address_h3_point) AS distance
        FROM rx_patients_with_h3 p
        JOIN geisinger.reference_data.geisinger_facilities f 
            ON p.address_h3_point IS NOT NULL 
            AND f.address_h3_point IS NOT NULL
            AND f.address_type = 'Pharmacy'
            AND f.address_closed = 'N'
            AND f.address_id != 'Specialty Pharmacy'
        WHERE h3_try_distance(p.address_h3_point, f.address_h3_point) <= 122
    ),
    rx_ranked_facilities AS (
        SELECT 
            person_id, facility_address_id, address_h3_point, distance,
            ROW_NUMBER() OVER (PARTITION BY person_id ORDER BY distance ASC) AS rank
        FROM rx_facilities_with_distance
    ),
    within_25m_of_pharmacy_cte AS (
        SELECT 
            person_id,
            "Y" as within_25m_of_pharmacy
        FROM rx_ranked_facilities
        WHERE rank = 1
        GROUP BY person_id
    )
    """,
    join_sql="LEFT JOIN within_25m_of_pharmacy_cte g17cte ON patients.person_id = g17cte.person_id",
)

processor.add_placeholder(
    "geisinger_retail_user",
    'coalesce(g18cte.geisinger_retail_user, "N") AS geisinger_retail_user',
    cte_sql=f"""
        geisinger_retail_user_cte AS (
            SELECT 
            person_id,
            CASE 
                WHEN COUNT(CASE WHEN fill_pharmacy = 'GEISINGER RETAIL PHARMACY' THEN 1 END) > 0 THEN 'Y' ELSE 'N' 
            END AS geisinger_retail_user
        FROM {gbl_target}medications
        GROUP BY person_id
        )
    """,
    join_sql="LEFT JOIN geisinger_retail_user_cte g18cte ON patients.person_id = g18cte.person_id",
)

processor.add_placeholder(
    "lapsed_retail_user",
    'coalesce(g19cte.lapsed_retail_user, "Y") AS lapsed_retail_user',
    cte_sql=f"""
        recent_retail_medications_lapsed AS (
            SELECT 
                m.person_id,
                COUNT(1) AS recent_fill_count
            FROM {gbl_target}medications m
            WHERE m.fill_pharmacy = 'GEISINGER RETAIL PHARMACY'
                AND m.rx_date_filled BETWEEN DATE_SUB('{max_encounter_date}', 180) AND '{max_encounter_date}'
            GROUP BY m.person_id
        ),
        prior_retail_medications_lapsed AS (
            SELECT 
                m.person_id,
                COUNT(1) AS prior_fill_count
            FROM {gbl_target}medications m
            WHERE 
                m.fill_pharmacy = 'GEISINGER RETAIL PHARMACY'
                AND m.rx_date_filled < DATE_SUB('{max_encounter_date}', 180)
            GROUP BY m.person_id
        ),
        lapsed_retail_user_cte AS (
            SELECT 
                r.person_id,
                CASE WHEN COALESCE(r.recent_fill_count, 0) = 0 AND COALESCE(p.prior_fill_count, 0) > 0 THEN 'Y' ELSE 'N' 
                END AS lapsed_retail_user
            FROM recent_retail_medications_lapsed r 
            LEFT JOIN prior_retail_medications_lapsed p 
            ON r.person_id = p.person_id
        )
    """,
    join_sql="LEFT JOIN lapsed_retail_user_cte g19cte ON patients.person_id = g19cte.person_id",
)

processor.add_placeholder(
    "distance_to_closest_pharmacy",
    "coalesce(g20cte.distance_to_closest_pharmacy, 245) as distance_to_closest_pharmacy",
    None,
    None,
)

processor.add_placeholder(
    "closest_pharmacy_name",
    "g20cte.closest_pharmacy_name",
    cte_sql=f"""
        patients_address_id_pharm AS (
        SELECT person_id, 
            sha1(concat_ws('|',
                coalesce(lower(person_mailing_add_line_1), ''), 
                coalesce(lower(person_mailing_add_line_2), ''),
                coalesce(lower(person_mailing_city), ''), 
                coalesce(lower(person_mailing_state), ''),
                coalesce(person_mailing_zip, '')
                )) AS address_id
        FROM {gbl_target}patients
    ), 
    patients_with_h3_pharm AS (
        SELECT a.*, b.address_h3_point
        FROM patients_address_id_pharm a
        JOIN geisinger.reference_data.geisinger_address b 
            ON a.address_id = b.address_id
    ),
    pharmacy_with_h3 AS (
        SELECT  
            CASE WHEN n.provider_practice_add_line_1 is null THEN
                sha1(concat_ws('|',
                coalesce(lower(n.provider_business_add_line_1), ''), 
                coalesce(lower(n.provider_business_add_line_2), ''),
                coalesce(lower(n.provider_business_add_city), ''), 
                coalesce(lower(n.provider_business_add_state), ''),
                coalesce(left(n.provider_business_add_postal_code, 5), '')))
            ELSE
                sha1(concat_ws('|',
                coalesce(lower(n.provider_practice_add_line_1), ''), 
                coalesce(lower(n.provider_practice_add_line_2), ''),
                coalesce(lower(n.provider_practice_add_city), ''), 
                coalesce(lower(n.provider_practice_add_state), ''),
                coalesce(left(n.provider_practice_add_postal_code, 5), '')))
            END AS address_id,
            n.provider_organization_name,
            na.address_h3_point
        FROM data_science.reference_data.nppes n
        JOIN data_science.reference_data.nppes_address na 
            ON
            CASE WHEN n.provider_practice_add_line_1 is null THEN
                sha1(concat_ws('|',
                coalesce(lower(n.provider_business_add_line_1), ''), 
                coalesce(lower(n.provider_business_add_line_2), ''),
                coalesce(lower(n.provider_business_add_city), ''), 
                coalesce(lower(n.provider_business_add_state), ''),
                coalesce(left(n.provider_business_add_postal_code, 5), '')))
            ELSE
                sha1(concat_ws('|',
                coalesce(lower(n.provider_practice_add_line_1), ''), 
                coalesce(lower(n.provider_practice_add_line_2), ''),
                coalesce(lower(n.provider_practice_add_city), ''), 
                coalesce(lower(n.provider_practice_add_state), ''),
                coalesce(left(n.provider_practice_add_postal_code, 5), '')))
            END = na.address_id
        WHERE (n.taxonomy_1 in ('333600000X', '3336C0002X', '3336C0003X', '3336C0004X', '3336I0012X', '183500000X') OR
               n.taxonomy_2 in ('333600000X', '3336C0002X', '3336C0003X', '3336C0004X', '3336I0012X', '183500000X') OR
               n.taxonomy_3 in ('333600000X', '3336C0002X', '3336C0003X', '3336C0004X', '3336I0012X', '183500000X')) AND
            n.provider_practice_add_state in ('PA', 'NY', 'NJ', 'OH', 'WV', 'DE', 'MD') AND
            n.provider_practice_add_line_1 is not null AND
            n.entity_type_code = 2    
    ),
    pharmacies_with_distance AS (
       SELECT 
            p.person_id, 
            f.provider_organization_name,
            p.address_h3_point as person_h3_point,
            f.address_h3_point as pharmacy_h3_point,
            h3_try_distance(p.address_h3_point, f.address_h3_point) AS distance
        FROM patients_with_h3_pharm p
        JOIN pharmacy_with_h3 f
        WHERE h3_try_distance(p.address_h3_point, f.address_h3_point) < 244 -- within 50 miles
    ),
    ranked_pharmacies AS (
        SELECT 
            person_id, provider_organization_name, person_h3_point, pharmacy_h3_point, distance,
            ROW_NUMBER() OVER (PARTITION BY person_id ORDER BY distance ASC) AS rank
        FROM pharmacies_with_distance
    ),
    closest_pharmacy_cte AS (
        SELECT 
            person_id,
            MAX(provider_organization_name) AS closest_pharmacy_name,
            MAX(distance) AS distance_to_closest_pharmacy
        FROM ranked_pharmacies
        WHERE rank = 1
        GROUP BY person_id
    )
    """,
    join_sql="LEFT JOIN closest_pharmacy_cte g20cte ON patients.person_id = g20cte.person_id",
)

processor.add_placeholder(
    "has_experienced_emergent_condition",
    'coalesce(g21cte.has_emergent_condition, "N") AS has_experienced_emergent_condition',
    cte_sql=f"""
        expanded_emergent_icd_10 AS (
            SELECT 
                category_code,
                category_description,
                TRIM(icd_10_code) AS icd_10_code
            FROM data_science.reference_data.emergent_ICD_10
            LATERAL VIEW explode(split(regexp_replace(icd_10_codes, '[\\\\[\\\\]\\']', ''), ',\\\\s*')) AS icd_10_code  
        ),
        has_emergent_condition_cte AS (
        SELECT 
            d.person_id,
            'Y' AS has_emergent_condition 
        FROM {gbl_target}diagnoses d
        JOIN expanded_emergent_icd_10 e
        ON regexp_replace(d.diagnosis_code, '\\\\.', '') = e.icd_10_code  
        GROUP BY d.person_id
        ORDER BY d.person_id
        )    
    """,
    join_sql="LEFT JOIN has_emergent_condition_cte g21cte ON patients.person_id = g21cte.person_id",
)

processor.add_placeholder(
    "has_experienced_non_emergent_condition",
    'coalesce(g22cte.has_subacute_condition, "N") AS has_experienced_non_emergent_condition',
    cte_sql=f"""
        expanded_subacute_icd_10 AS (
            SELECT 
                category_code,
                category_description,
                TRIM(icd_10_code) AS icd_10_code
            FROM data_science.reference_data.subacute_ICD_10
            LATERAL VIEW explode(split(regexp_replace(icd_10_codes, '[\\\\[\\\\]\\']', ''), ',\\\\s*')) AS icd_10_code  
        ),
        has_subacute_condition_cte AS (
        SELECT 
            d.person_id,
            'Y' AS has_subacute_condition 
        FROM {gbl_target}diagnoses d
        JOIN expanded_subacute_icd_10 e
        ON regexp_replace(d.diagnosis_code, '\\\\.', '') = e.icd_10_code  
        GROUP BY d.person_id
        ORDER BY d.person_id
        )    
    """,
    join_sql="LEFT JOIN has_subacute_condition_cte g22cte ON patients.person_id = g22cte.person_id",
)

processor.add_placeholder(
    "short_term_medications",
    'coalesce(g23cte.short_term_medications, "N") as short_term_medications',
    cte_sql=f"""
        short_term_medications_cte AS (
            SELECT person_id, CASE WHEN count(*) > 0 THEN 'Y' ELSE 'N' END AS short_term_medications
            FROM {gbl_target}medications
            JOIN geisinger.reference_data.short_term_drug_classification ON medications.drug_classification = short_term_drug_classification.drug_classification
            group by person_id
            )
    """,
    join_sql="LEFT JOIN short_term_medications_cte g23cte ON patients.person_id = g23cte.person_id",
)

processor.add_placeholder(
    "valve_group",
    f"""
    coalesce(valve_group_a, "N") AS valve_group_a,
    coalesce(valve_group_b, "N") AS valve_group_b
    """,
    cte_sql=f"""
        patient_criteria AS (
            SELECT
                person_id,
                CASE
                    WHEN SUM(CASE WHEN IPCV_Finding_Code = 'Severe aortic valve stenosis is present.' THEN 1 ELSE 0 END) > 0
                    THEN 'Y'
                    ELSE 'N'
                END AS valve_group_a,
                CASE
                    WHEN 
                        SUM(CASE WHEN IPCV_Finding_Code = 'Severe aortic valve stenosis is present.' THEN 1 ELSE 0 END) = 0
                        AND SUM(CASE WHEN IPCV_Finding_Code = 'Moderate aortic valve stenosis is present.' THEN 1 ELSE 0 END) > 0
                    THEN 'Y'
                    ELSE 'N'
                END AS valve_group_b
            FROM {gbl_target}diagnoses
            GROUP BY person_id
        )
    """,
    join_sql="LEFT JOIN patient_criteria valve_cte ON patients.person_id = valve_cte.person_id",
)

processor.add_placeholder(
    "elix_hauser_score",
    "COALESCE(ehscte.elixhauser_score, 0) AS elixhauser_score",
    cte_sql=f"""
        distinct_diagnoses AS (
        SELECT 
            person_id,
            UPPER(REGEXP_REPLACE(diagnosis_code, '\\\\.', '')) AS clean_icd
        FROM {gbl_target}diagnoses
        GROUP BY person_id, diagnosis_code
        ),
        mapped_comorbidities AS (
        SELECT 
            d.person_id,
            r.*
        FROM distinct_diagnoses d
        JOIN data_science.reference_data.cmr_reference_file_v_2025_1 r 
            ON d.clean_icd = r.icd_10_cm_code
        ),
        pivoted_flags_cte AS (
        SELECT
            person_id,
            MAX(AIDS) AS elix_aids,
            MAX(ALCOHOL) AS elix_alcohol,
            MAX(ANEMDEF) AS elix_anemdef,
            MAX(AUTOIMMUNE) AS elix_autoimmune,
            MAX(BLDLOSS) AS elix_bldloss,
            MAX(CANCER_LEUK) AS elix_cancer_leuk,
            MAX(CANCER_LYMPH) AS elix_cancer_lymph,
            MAX(CANCER_METS) AS elix_cancer_mets,
            MAX(CANCER_NSITU) AS elix_cancer_nsitu,
            MAX(CANCER_SOLID) AS elix_cancer_solid,
            MAX(CBVD_POA) AS elix_cbvd_poa,
            MAX(CBVD_SQLA) AS elix_cbvd_sqla,
            MAX(COAG) AS elix_coag,
            MAX(DEMENTIA) AS elix_dementia,
            MAX(DEPRESS) AS elix_depress,
            MAX(DIAB_CX) AS elix_diab_cx,
            MAX(DIAB_UNCX) AS elix_diab_uncx,
            MAX(DRUG_ABUSE) AS elix_drug_abuse,
            MAX(HF) AS elix_hf,
            MAX(HTN_CX) AS elix_htn_cx,
            MAX(HTN_UNCX) AS elix_htn_uncx,
            MAX(LIVER_MLD) AS elix_liver_mld,
            MAX(LIVER_SEV) AS elix_liver_sev,
            MAX(LUNG_CHRONIC) AS elix_lung_chronic,
            MAX(NEURO_MOVT) AS elix_neuro_movt,
            MAX(NEURO_OTH) AS elix_neuro_oth,
            MAX(NEURO_SEIZ) AS elix_neuro_seiz,
            MAX(OBESE) AS elix_obese,
            MAX(PARALYSIS) AS elix_paralysis,
            MAX(PERIVASC) AS elix_perivasc,
            MAX(PSYCHOSES) AS elix_psychoses,
            MAX(PULMCIRC) AS elix_pulmcirc,
            MAX(RENLFL_MOD) AS elix_renlfl_mod,
            MAX(RENLFL_SEV) AS elix_renlfl_sev,
            MAX(THYROID_HYPO) AS elix_thyroid_hypo,
            MAX(THYROID_OTH) AS elix_thyroid_oth,
            MAX(ULCER_PEPTIC) AS elix_ulcer_peptic,
            MAX(VALVE) AS elix_valve,
            MAX(WGHTLOSS) AS elix_wghtloss
        FROM mapped_comorbidities
        GROUP BY person_id
        ),
        elix_score_cte AS (
        SELECT person_id,
            -- Total score as count of flags
            elix_aids + elix_alcohol + elix_anemdef + elix_autoimmune + elix_bldloss +
            elix_cancer_leuk + elix_cancer_lymph + elix_cancer_mets + elix_cancer_nsitu + elix_cancer_solid +
            elix_cbvd_poa + elix_cbvd_sqla + elix_coag + elix_dementia + elix_depress + elix_diab_cx + elix_diab_uncx +
            elix_drug_abuse + elix_hf + elix_htn_cx + elix_htn_uncx + elix_liver_mld + elix_liver_sev +
            elix_lung_chronic + elix_neuro_movt + elix_neuro_oth + elix_neuro_seiz + elix_obese + elix_paralysis +
            elix_perivasc + elix_psychoses + elix_pulmcirc + elix_renlfl_mod + elix_renlfl_sev +
            elix_thyroid_hypo + elix_thyroid_oth + elix_ulcer_peptic + elix_valve + elix_wghtloss AS elixhauser_score
        FROM pivoted_flags_cte
        )
    """,
    join_sql=f"""
        LEFT JOIN pivoted_flags_cte ehpvfcte ON patients.person_id = ehpvfcte.person_id
        LEFT JOIN elix_score_cte ehscte ON patients.person_id = ehscte.person_id
    """,
)

processor.add_placeholder(
    "elix_hauser_flags",
    f"""
        COALESCE(ehpvfcte.elix_aids, 0) AS elix_aids,
        COALESCE(ehpvfcte.elix_alcohol, 0) AS elix_alcohol,
        COALESCE(ehpvfcte.elix_anemdef, 0) AS elix_anemdef,
        COALESCE(ehpvfcte.elix_autoimmune, 0) AS elix_autoimmune,
        COALESCE(ehpvfcte.elix_bldloss, 0) AS elix_bldloss,
        COALESCE(ehpvfcte.elix_cancer_leuk, 0) AS elix_cancer_leuk,
        COALESCE(ehpvfcte.elix_cancer_lymph, 0) AS elix_cancer_lymph,
        COALESCE(ehpvfcte.elix_cancer_mets, 0) AS elix_cancer_mets,
        COALESCE(ehpvfcte.elix_cancer_nsitu, 0) AS elix_cancer_nsitu,
        COALESCE(ehpvfcte.elix_cancer_solid, 0) AS elix_cancer_solid,
        COALESCE(ehpvfcte.elix_cbvd_poa, 0) AS elix_cbvd_poa,
        COALESCE(ehpvfcte.elix_cbvd_sqla, 0) AS elix_cbvd_sqla,
        COALESCE(ehpvfcte.elix_coag, 0) AS elix_coag,
        COALESCE(ehpvfcte.elix_dementia, 0) AS elix_dementia,
        COALESCE(ehpvfcte.elix_depress, 0) AS elix_depress,
        COALESCE(ehpvfcte.elix_diab_cx, 0) AS elix_diab_cx,
        COALESCE(ehpvfcte.elix_diab_uncx, 0) AS elix_diab_uncx,
        COALESCE(ehpvfcte.elix_drug_abuse, 0) AS elix_drug_abuse,
        COALESCE(ehpvfcte.elix_hf, 0) AS elix_hf,
        COALESCE(ehpvfcte.elix_htn_cx, 0) AS elix_htn_cx,
        COALESCE(ehpvfcte.elix_htn_uncx, 0) AS elix_htn_uncx,
        COALESCE(ehpvfcte.elix_liver_mld, 0) AS elix_liver_mld,
        COALESCE(ehpvfcte.elix_liver_sev, 0) AS elix_liver_sev,
        COALESCE(ehpvfcte.elix_lung_chronic, 0) AS elix_lung_chronic,
        COALESCE(ehpvfcte.elix_neuro_movt, 0) AS elix_neuro_movt,
        COALESCE(ehpvfcte.elix_neuro_oth, 0) AS elix_neuro_oth,
        COALESCE(ehpvfcte.elix_neuro_seiz, 0) AS elix_neuro_seiz,
        COALESCE(ehpvfcte.elix_obese, 0) AS elix_obese,
        COALESCE(ehpvfcte.elix_paralysis, 0) AS elix_paralysis,
        COALESCE(ehpvfcte.elix_perivasc, 0) AS elix_perivasc,
        COALESCE(ehpvfcte.elix_psychoses, 0) AS elix_psychoses,
        COALESCE(ehpvfcte.elix_pulmcirc, 0) AS elix_pulmcirc,
        COALESCE(ehpvfcte.elix_renlfl_mod, 0) AS elix_renlfl_mod,
        COALESCE(ehpvfcte.elix_renlfl_sev, 0) AS elix_renlfl_sev,
        COALESCE(ehpvfcte.elix_thyroid_hypo, 0) AS elix_thyroid_hypo,
        COALESCE(ehpvfcte.elix_thyroid_oth, 0) AS elix_thyroid_oth,
        COALESCE(ehpvfcte.elix_ulcer_peptic, 0) AS elix_ulcer_peptic,
        COALESCE(ehpvfcte.elix_valve, 0) AS elix_valve,
        COALESCE(ehpvfcte.elix_wghtloss, 0) AS elix_wghtloss
    """,
    None,
    None,
)

processor.add_placeholder(
    "ghp_rx_benefit",
    'coalesce(ghprx.ghp_rx_benefit, "N") as ghp_rx_benefit',
    cte_sql=f"""
        ghp_rx_benefit_cte AS (
            SELECT
                ins.person_id,
                ins.plan_name,
                ins.plan_id,
                ic.plan_category,
                'Y' as ghp_rx_benefit
            FROM {gbl_target}insurance ins
            LEFT JOIN `geisinger`.`reference_data`.`insurance_category` ic ON ins.plan_id = ic.plan_id
            WHERE ic.plan_category in ('GHP Gold', 'GHP Commercial')
        )
    """,
    join_sql="LEFT JOIN ghp_rx_benefit_cte ghprx ON patients.person_id = ghprx.person_id",
)

processor.add_placeholder(
    "recent_mail_order_user",
    'coalesce(rmocte.recent_mail_order_user, "N") AS recent_mail_order_user',
    cte_sql=f"""
        recent_mail_order_user_cte AS (
            SELECT
                person_id,
                fill_pharmacy as mail_order_fill_pharmacy,
                rx_date_filled as mail_order_rx_date_filled,
                'Y' AS recent_mail_order_user
            FROM {gbl_target}medications
            WHERE fill_pharmacy = 'GEISINGER MAIL ORDER PHARMACY'
            AND DATE("2025-12-19") - INTERVAL '18 months' <= rx_date_filled
        )
    """,
    join_sql="LEFT JOIN recent_mail_order_user_cte rmocte ON patients.person_id = rmocte.person_id",
)

processor.add_placeholder(
    "recent_retail_user",
    'coalesce(rrcte.recent_retail_user, "N") AS recent_retail_user',
    cte_sql=f"""
        recent_retail_user_cte AS (
            SELECT
                person_id,
                fill_pharmacy as retail_fill_pharmacy,
                rx_date_filled as retail_rx_date_filled,
                'Y' AS recent_retail_user
            FROM {gbl_target}medications
            WHERE fill_pharmacy = 'GEISINGER RETAIL PHARMACY'
            AND DATE("2025-12-19") - INTERVAL '18 months' <= rx_date_filled
        )
    """,
    join_sql="LEFT JOIN recent_retail_user_cte rrcte ON patients.person_id = rrcte.person_id",
)

In [0]:
#   ,add.address_latitude as address_latitude_patient
#   ,add.address_longitude as address_longitude_patient
#   ,fac.address_id as address_id_65f
#   ,fac.address_latitude as address_latitude_65f
#   ,fac.address_longitude as address_longitude_65f
#   ,2 * 3958.8 * ASIN(
#     SQRT(
#       POWER(SIN(RADIANS(fac.address_latitude - add.address_latitude) / 2), 2) +
#       COS(RADIANS(add.address_latitude)) * COS(RADIANS(fac.address_latitude)) *
#       POWER(SIN(RADIANS(fac.address_longitude - add.address_longitude) / 2), 2)
#     )
#   ) AS address_patient_65f_distance_miles

# left join geisinger.reference_data.geisinger_address add
#   on sha1(concat_ws('|',
#             coalesce(lower(pat.person_mailing_add_line_1), ''),
#             coalesce(lower(pat.person_mailing_add_line_2), ''),
#             coalesce(lower(pat.person_mailing_city), ''),
#             coalesce(lower(pat.person_mailing_state), ''),
#             coalesce(pat.person_mailing_zip, '')
#         )) = add.address_id

# select *
# from geisinger.reference_data.geisinger_facilities
# where 1=1
#   and 65_forward_facility = 'Y'
#   and address_closed = 'N'
#   and address_id != '65 Forward Health Center Milton' --Exclusion based on website snapshot as of 2025-11-04 (only 10 open at that time): https://www.geisinger.org/patient-care/65-forward/location-listing
# )